## Setup
Here we will import the libraries that are needed, and load the data into a pandas dataframe.
Also, the OpenAI client will be set up and authentication is handled.

In [10]:
import pandas as pd
import os
import json
from openai import OpenAI

# Load the CSV file
df = pd.read_csv('./dataset.csv')

# Limit the number of rows (Set to None to process all rows)
row_limit = 100  # Adjust this number based on cost considerations

client = OpenAI(
    api_key='<API_KEY>'
)

### Prompt

In [8]:
chain_of_thought = """
"Given an email, your task is to analyze the email to determine if it's a phishing attempt. Apply a chain-of-thought process by following this detailed framework, which includes psychological traits and cognitive biases, complete with explanations and examples. For the email, proceed through the steps outlined below, utilizing your critical thinking and knowledge to guide your analysis and conclusions.

Step 1: Identify Psychological Traits and Cognitive Biases

Before analyzing the emails, understand these psychological traits and cognitive biases that phishers exploit, along with explanations and specific examples:

    A Sense of Urgency
        Explanation: Creating a sense of urgency pressures the recipient into making decisions quickly, often without sufficient time to consider the consequences.
        Examples:
            Email claiming account suspension within 24 hours unless a link is clicked.
            Notification of an outstanding invoice requiring immediate payment.

    Inducing Fear by Threatening
        Explanation: This involves invoking fear to coerce individuals into complying with the attacker's demands, under the threat of negative outcomes.
        Examples:
            Message claiming to be from the tax authority threatens legal action and fines for supposedly unpaid taxes.
            Email falsely claiming that personal data has been compromised in a security breach.

    Enticement with Desire
        Explanation: This tactic plays on desires or greed, offering something too good to be true to lure individuals into performing specific actions.
        Examples:
            Email congratulating on winning a large sum of money, requesting personal information or a fee to process the winnings.
            Message offering an exclusive deal or a significant discount on high-value products.

    Authority Bias
        Explanation: This bias makes individuals more likely to trust and follow suggestions or orders from a figure of authority or someone perceived as an expert.
        Examples:
            Email purportedly from the company's CEO directs the employee to transfer funds urgently or share sensitive information.
            Communication claiming to be from a government agency, demanding personal information for verification purposes.

    Recency Effect
        Explanation: This effect describes the tendency to remember and prioritize the most recently presented information over older data.
        Examples:
            Phishing campaign following a natural disaster, urging donations to a fraudulent charity.
            Emails advertising a recent technological breakthrough, offering early access in exchange for personal details or financial information.

    Halo Effect
        Explanation: The halo effect occurs when an overall impression of a person or brand influences feelings and thoughts about their character or properties.
        Examples:
            Email mimicking a well-respected brand, such as Apple or Microsoft, to trick individuals into disclosing login credentials.
            Messages claiming a celebrity endorses a product or investment scheme, urging immediate action to take advantage of the offer.

    Hyperbolic Discounting
        Explanation: This describes the preference for smaller, immediate rewards over larger, delayed benefits, exploiting the human tendency to want instant gratification.
        Examples:
            Email offering an immediate but limited-time discount, requiring quick action to provide payment information.
            Message promising a high return on investment in a short period, asking for an upfront payment or financial details.

    Curiosity Effect
        Explanation: This effect leverages human curiosity to entice individuals into seeking more information, often leading them into risky situations.
        Examples:
            Email with a vague but intriguing subject line, tempting the recipient to open an attached file that contains malware.
            Message that piques curiosity with a vague promise of a unique opportunity, directing users to a phishing site designed to harvest personal data.

This comprehensive understanding of psychological traits and cognitive biases will serve as a critical foundation for your email analysis. Keep these traits and examples in mind as you examine each email in the dataset, looking for signs that indicate the use of these tactics.

Step 2: Examine The Email

For every email, look for indications of these traits and biases. Consider:

    Does the email create an urgent need to act?
    Is there a threat that induces fear?
    Are there offers that entice with desire or seem too good to be true?
    Does the email falsely claim authority or expertise?
    Are recent events exploited to make the message seem more urgent or relevant?
    Is there an implication of a reputable brand or celebrity endorsement to gain trust?
    Are there offers for immediate rewards?
    Does the email pique curiosity with vague promises or intriguing attachments?

Step 3: Implementation Description

Describe how each identified trait or bias is implemented within the email. This involves considering:

    The language and tone used: Is it pressing, threatening, overly enticing, or authoritative?
    The context: Are there references to current events, reputable brands, or authorities?
    The offer: Is there a promise of immediate gain or a solution to a provoked fear or problem?

Step 4: Apply Chain-of-Thought Reasoning

Reflect on your findings and apply reasoning:

    If multiple traits and biases are present, how do they interact?
    Do the specific examples provided in the framework match the tactics used in the email?
    Considering the overall context and presentation, does the email align with common phishing strategies?

Step 5: Make a Conclusion

Based on the analysis, conclude whether the email is likely a phishing attempt or not. Consider:

    The number and nature of phishing characteristics identified.
    How closely the email follows known phishing patterns and examples.
    Any external knowledge or phishing indicators, such as suspicious sender addresses or requests for personal information.

Reflecting on the Analysis Process

This chain-of-thought approach encourages a comprehensive examination of each email, ensuring no detail is overlooked. By methodically considering each step and applying reasoned judgment, you'll be better equipped to identify and understand phishing attempts within your dataset.

Instructions: provide the analysis result in json format with the following parts:
is_deceptive: Boolean (True for phishing, False for not phishing)
explanation: Text explaining the reasoning behind the decision."
"""

## Analysis
In this part, every email in the provided dataset is analyzed to determine if it is a phishing email or not. 
The results are then saved to another csv file for further analysis.

In [11]:
# Create a new DataFrame to store the processed rows
processed_df = pd.DataFrame(columns=df.columns)

# Process each row in the DataFrame
for index, row in df.iterrows():
    if row_limit is not None and index >= row_limit:
        break

    email_text = row['text']    

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": chain_of_thought},
            {"role": "user", "content": email_text}
        ]
    )
    response = completion.choices[0].message.content
    response_json = json.loads(response)

    new_row = pd.DataFrame({
        'text': [email_text],
        'is_deceptive': [response_json['is_deceptive']],
        'explanation': [response_json['explanation']],
    })
    processed_df = pd.concat([processed_df, new_row], ignore_index=True)

# Export DataFrame to a new CSV file
processed_df.to_csv('processed.csv', index=False)

## Statistical performance analysis
In this part, the performance of the model is analyzed using statistical measures such as accuracy, precision, recall, and F1 score.

In [3]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the datasets
processed_df = pd.read_csv('./processed.csv')
true_labels_df = pd.read_csv('./sample_labeled.csv')

# Extracting the predicted and true labels
y_pred = processed_df['is_deceptive'].values
y_true = true_labels_df['is_deceptive'].values

# Calculate the evaluation metrics
gpt3_accuracy = accuracy_score(y_true, y_pred)
gpt3_precision = precision_score(y_true, y_pred)
gpt3_recall = recall_score(y_true, y_pred)
gpt3_f1 = f1_score(y_true, y_pred)

# Print the scores
print(f"Accuracy: {gpt3_accuracy*100:.2f}%")
print(f"Precision: {gpt3_precision*100:.2f}%")
print(f"Recall: {gpt3_recall*100:.2f}%")
print(f"F1 Score: {gpt3_f1*100:.2f}%")


Accuracy: 80.00%
Precision: 70.15%
Recall: 100.00%
F1 Score: 82.46%


1. Accuracy (80.00%)

    Implication: With an accuracy of 80%, the system correctly identifies 8 out of every 10 emails, whether they're phishing or legitimate. This level suggests that the system has a good foundation but might be improved to increase its reliability, especially in detecting legitimate emails accurately.

2. Precision (70.15%)

    Implication: A precision of 70.15% indicates that when the system flags an email as phishing, there's a roughly 70% chance it is indeed phishing. This lower precision compared to the previous scenarios suggests a relatively higher rate of false positives, where legitimate emails are misclassified as phishing. This could disrupt user experience and trust, highlighting a need for improvement in distinguishing phishing emails more accurately.

3. Recall (100.00%)

    Implication: A recall of 100% is noteworthy, indicating the system identifies all actual phishing attempts without missing any. This perfect recall rate is crucial for security, ensuring no phishing emails bypass the system to reach end-users. However, the high recall, in this case, may be contributing to the lower precision, suggesting the system errs on the side of caution by flagging more emails as phishing, including some that are not.

4. F1 Score (82.46%)

    Implication: The F1 score, at 82.46%, reflects a balance between precision and recall, weighted more heavily towards recall due to its perfect score. While the high F1 score suggests the system is effective overall, the balance is skewed, indicating that the system might be overzealous in its phishing detection, leading to more false positives.
    
In the Context of Phishing Countermeasures:

Prioritizing Security Over Convenience: The perfect recall rate demonstrates a prioritization of security, aiming to ensure no phishing emails reach users. However, this comes at the cost of user convenience due to a higher false positive rate.

Need for Refinement to Reduce False Positives: To improve the user experience and maintain trust in the system, efforts should focus on reducing false positives. This might involve more sophisticated or tailored detection algorithms that can more accurately distinguish between phishing and legitimate emails.

A Trade-off Between Precision and Recall: This scenario exemplifies a trade-off where maximizing recall has led to diminished precision. Optimizing both metrics simultaneously can be challenging but is essential for a balanced and effective phishing detection system.

Importance of Continuous Monitoring and Feedback: Incorporating continuous monitoring and feedback mechanisms can help identify areas for improvement, particularly in reducing false positives. User feedback can also provide valuable insights into the system's performance in real-world scenarios.

Complementary Measures: Given the system's current limitations, complementary security measures and user education become even more critical. Educating users about phishing and encouraging vigilance can serve as an additional layer of defense, especially in scenarios where legitimate emails are misclassified.

# GPT-4 Analysis

In [16]:
# Create a new DataFrame to store the processed rows
processed_df = pd.DataFrame(columns=df.columns)

# Process each row in the DataFrame
for index, row in df.iterrows():
    if row_limit is not None and index >= row_limit:
        break
    # Construct the prompt
    prompt = chain_of_thought + f"\n\n{row['text']}"    
    # print("Prompt:", prompt)

    email_text = row['text']    

    completion = client.chat.completions.create(
        model="gpt-4-turbo-preview",
        # model="gpt-3.5-turbo-1106",
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": chain_of_thought},
            {"role": "user", "content": email_text}
        ]
    )
    response = completion.choices[0].message.content
    response_json = json.loads(response)

    new_row = pd.DataFrame({
        'text': [email_text],
        'is_deceptive': [response_json['is_deceptive']],
        'explanation': [response_json['explanation']],
    })
    processed_df = pd.concat([processed_df, new_row], ignore_index=True)


# Export DataFrame to a new CSV file
processed_df.to_csv('gpt4t-cot-processed.csv', index=False)

In [17]:
a = processed_df

In [8]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the datasets
processed_df = pd.read_csv('./gpt4t-cot-processed.csv')
true_labels_df = pd.read_csv('./sample_labeled.csv')

# Make sure the datasets are aligned if necessary. This example assumes they're already aligned.

# Extracting the predicted and true labels
y_pred = processed_df['is_deceptive'].values
y_true = true_labels_df['is_deceptive'].values

# Calculate the evaluation metrics
gpt4_accuracy = accuracy_score(y_true, y_pred)
gpt4_precision = precision_score(y_true, y_pred)
gpt4_recall = recall_score(y_true, y_pred)
gpt4_f1 = f1_score(y_true, y_pred)

# Print the scores
print(f"Accuracy: {gpt4_accuracy*100:.2f}%")
print(f"Precision: {gpt4_precision*100:.2f}%")
print(f"Recall: {gpt4_recall*100:.2f}%")
print(f"F1 Score: {gpt4_f1*100:.2f}%")


Accuracy: 91.00%
Precision: 93.18%
Recall: 87.23%
F1 Score: 90.11%


In [9]:
import plotly.graph_objects as go

# Data for GPT-3 and GPT-4
models = ['GPT-3', 'GPT-4']
accuracy = [gpt3_accuracy, gpt4_accuracy]
precision = [gpt3_precision, gpt4_precision]
recall = [gpt3_recall, gpt4_recall]
f1_score = [gpt3_f1, gpt4_f1]

# Create subplots
fig = go.Figure()

# Add bar for each metric
fig.add_trace(go.Bar(x=models, y=accuracy, name='Accuracy', marker_color='indianred'))
fig.add_trace(go.Bar(x=models, y=precision, name='Precision', marker_color='lightsalmon'))
fig.add_trace(go.Bar(x=models, y=recall, name='Recall', marker_color='lightseagreen'))
fig.add_trace(go.Bar(x=models, y=f1_score, name='F1 Score', marker_color='royalblue'))

# Update layout for clarity
fig.update_layout(
    title='Comparison of GPT-3 and GPT-4 Performance Metrics',
    xaxis=dict(
        title='Model',
        tickmode='linear',
    ),
    yaxis=dict(
        title='Percentage (%)',
        range=[88, 94]
    ),
    barmode='group',
    legend=dict(
        title='Metrics'
    )
)

# Save the figure
fig.show()

## Other Datasets

In [ ]:
df = pd.read_csv('./dataset.csv')
# Create a new DataFrame to store the processed rows
processed_df = pd.DataFrame(columns=df.columns)

# Process each row in the DataFrame
for index, row in df.iterrows():
    if row_limit is not None and index >= row_limit:
        break
    # Construct the prompt
    prompt = chain_of_thought + f"\n\n{row['text']}"    
    # print("Prompt:", prompt)

    email_text = row['text']    

    completion = client.chat.completions.create(
        model="gpt-4-turbo-preview",
        # model="gpt-3.5-turbo-1106",
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": chain_of_thought},
            {"role": "user", "content": email_text}
        ]
    )
    response = completion.choices[0].message.content
    response_json = json.loads(response)

    new_row = pd.DataFrame({
        'text': [email_text],
        'is_deceptive': [response_json['is_deceptive']],
        'explanation': [response_json['explanation']],
    })
    processed_df = pd.concat([processed_df, new_row], ignore_index=True)


# Export DataFrame to a new CSV file
processed_df.to_csv('gpt4t-cot-processed.csv', index=False)